In [1]:
# Initial Library imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
import numpy as np
import datetime as dt


In [2]:
# Load .env environment variables
load_dotenv()

True

In [3]:
# Set adjustable variables
no_tickers = 30
start_date = "2022-05-5" # format yyyy-mm-dd
end_date   = "2022-05-12" # format yyyy-mm-dd

In [4]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

In [5]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [6]:
# import csv list of all nasdaq tickers and clean them for use

#import csv file of all tickers to a dataframe
df_full_ticker_list = pd.read_csv ('Data/nasdaq_screener_assets.csv')
#drop unwanted columns
df_full_ticker_list = df_full_ticker_list.drop(['Last Sale', 'Net Change', '% Change', 'Market Cap',
       'Country', 'IPO Year', 'Volume', 'Sector', 'Industry'], axis = 'columns')

# Remove Tickers with '^', '/', & spaces as Alpaca does not accept them
df_full_ticker_list = df_full_ticker_list[~df_full_ticker_list.Symbol.str.contains('^', regex = False)
                                    & ~df_full_ticker_list.Symbol.str.contains('/', regex = False)
                                    & ~df_full_ticker_list.Symbol.str.contains(' ', regex = False)
                                   ]

# convert df to list for use with Alpaca API
alpaca_tickers = list(df_full_ticker_list['Symbol'])


In [7]:
#set up for Alpaca API call

# Format current date as ISO format
start_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()

#Set timeframe of Tickers 
timeframe = "1Day"


In [8]:
# make API call to Alpaca to receive a data frame of all stock data

df_stock_all = alpaca.get_bars(
    alpaca_tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

#check output of df_stock_all
df_stock_all.head(10)

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2022-05-05 04:00:00+00:00,67.77,68.1700,61.94,63.46,6033723,63532,63.650954,AA
2022-05-06 04:00:00+00:00,62.65,62.8000,60.00,61.04,5562796,53067,61.417683,AA
2022-05-09 04:00:00+00:00,58.50,58.5731,53.73,54.21,12052318,113405,55.454040,AA
2022-05-10 04:00:00+00:00,55.60,56.9100,53.61,55.84,8457915,89397,55.533974,AA
2022-05-11 04:00:00+00:00,57.46,59.3600,56.27,56.66,6162461,63078,57.584700,AA
2022-05-12 04:00:00+00:00,54.85,56.3600,53.34,54.58,7370674,72508,54.742692,AA
2022-05-05 04:00:00+00:00,9.83,9.8400,9.83,9.83,171648,173,9.834259,AAC
2022-05-06 04:00:00+00:00,9.83,9.8350,9.81,9.81,1712191,254,9.813244,AAC
2022-05-09 04:00:00+00:00,9.82,9.8350,9.81,9.82,60324,241,9.812899,AAC


In [9]:
# calculate the change in price accross the selected date range to determine a suitable dataset for analysis

# Set up df_stock_price_change varialbe as data frame to capture all start and end values
df_stock_price_change = pd.DataFrame(columns = ['Ticker', 'Start Price', 'End Price','Price Change'])
# capture all Ticker symbols in df_stock_price_change form df_full_ticker_list
df_stock_price_change['Ticker'] = df_full_ticker_list['Symbol']

# loop through each ticker and capture the first and last pice for each
for index in df_stock_price_change.index:
    ticker = df_stock_price_change['Ticker'][index]
    ticker_data_temp = df_stock_all[df_stock_all.symbol == ticker]
    
    # ignore ticker if no symbol was found from Alpaca, only process if the size of ticker_data_temp is greater than 0
    if ticker_data_temp.size != 0:
    
        df_stock_price_change['Start Price'][index] = ticker_data_temp.at[ticker_data_temp.index.max(), 'close']
        df_stock_price_change['End Price'][index] = ticker_data_temp.at[ticker_data_temp.index.min(), 'close']

# calculate price change
df_stock_price_change['Price Change'] = df_stock_price_change['End Price'] - df_stock_price_change['Start Price']
df_stock_price_change['Price Change %'] = (df_stock_price_change['Price Change']/df_stock_price_change['Start Price'])*100

# check output of df_stock_price_change
df_stock_price_change.head(10)

,Ticker,Start Price,End Price,Price Change,Price Change %
0,A,116.13,122.15,6.02,5.183846
1,AA,54.58,63.46,8.88,16.269696
2,AAC,9.78,9.83,0.05,0.511247
3,AACG,0.935,1.16,0.225,24.064171
4,AACI,9.86,9.86,0.0,0.0
5,AACIW,0.26,0.2709,0.0109,4.192308
6,AADI,12.65,15.71,3.06,24.189723
7,AAIC,2.9,3.19,0.29,10.0
10,AAIN,23.5437,23.75,0.2063,0.876243
11,AAL,15.49,18.54,3.05,19.690123


In [10]:
# Clean df_stock_price_change, drop NaNs and sort in order of % change
df_stock_price_change.sort_values(by = 'Price Change %',ascending = False, inplace = True)
df_stock_price_change.dropna(axis = 'index', how = 'any' ,inplace = True)
df_stock_price_change.reset_index(drop = True, inplace = True)

# check output of df_stock_price_change
df_stock_price_change.head(10)

,Ticker,Start Price,End Price,Price Change,Price Change %
0,AKAN,1.14,8.96,7.82,685.964912
1,TNON,8.86,36.28,27.42,309.480813
2,VIEWW,0.0491,0.1875,0.1384,281.873727
3,RDBXW,0.13,0.4398,0.3098,238.307692
4,VIEW,0.4687,1.51,1.0413,222.167698
5,CELUW,0.53,1.6,1.07,201.886792
6,ITP,0.1701,0.5,0.3299,193.944738
7,DAVEW,0.2,0.5725,0.3725,186.25
8,FTPAW,0.1844,0.5238,0.3394,184.056399
9,UPST,32.78,89.04,56.26,171.629042


In [11]:
# Select Tickers for analysis distributed through Data Set equally
# this is to ensure that the tickers selected for analysis are a well represented set across a wide market range 

#set number of rows
rows_count = df_stock_price_change.shape[0]
# calculate step size for selection less one to avoid the ends
increment = round(rows_count/(no_tickers))-1
# set indexes of tickers to capture for analysis
list_select_tickers = range(round(no_tickers/2), rows_count, increment)

# capture the ticker symbols to use for anlysis
list_analysis_tickers = list(df_stock_price_change['Ticker'][list_select_tickers].values)

# check output of list_analysis_tickers
list_analysis_tickers

['DBGIW',
 'NVIV',
 'LOTZ',
 'BRPMW',
 'GRTX',
 'VHC',
 'IMV',
 'CGEN',
 'COCO',
 'SISI',
 'PBFX',
 'EMBC',
 'SHYF',
 'AMP',
 'EPSN',
 'PNC',
 'CYTK',
 'EFT',
 'COLL',
 'PDEX',
 'PZC',
 'PEBK',
 'DISAU',
 'SCLE',
 'EQHA',
 'AGNC',
 'IHTA',
 'MIME',
 'SOJC',
 'DS',
 'METX']

In [ ]:
# capture stock price data of the selected tickers and clean ready for analysis

# capture the data for only the chosen tickers from df_stock_all
df_stock_price_data = df_stock_all[df_stock_all['symbol'].isin(list_analysis_tickers)]
# reset index
df_stock_price_data.reset_index(inplace = True)

# Change 'timestamp' values to date only
df_stock_price_data.loc[:,'timestamp'] = df_stock_price_data.loc[:,'timestamp'].dt.date

# drop un-needed columns
df_stock_price_data.drop(['open','high','low','volume','trade_count','vwap'], axis='columns',inplace = True)
# rename remaining columns to suitable names
df_stock_price_data.columns = ['Date','Close','Ticker']

# check output of df_stock_price_data
df_stock_price_data.head(10)

C:\Users\mclew\anaconda3\envs\alpacaenv\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\mclew\anaconda3\envs\alpacaenv\lib\site-packages\pandas\core\frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Date,Close,Ticker
0,2022-05-05 04:00:00+00:00,12.14,AGNC
1,2022-05-06 04:00:00+00:00,12.41,AGNC
2,2022-05-09 04:00:00+00:00,12.26,AGNC
3,2022-05-10 04:00:00+00:00,12.23,AGNC
4,2022-05-11 04:00:00+00:00,12.14,AGNC
5,2022-05-12 04:00:00+00:00,12.11,AGNC
6,2022-05-05 04:00:00+00:00,270.70,AMP
7,2022-05-06 04:00:00+00:00,266.60,AMP
8,2022-05-09 04:00:00+00:00,259.64,AMP
9,2022-05-10 04:00:00+00:00,259.39,AMP


In [13]:
# save CSV of df_stock_price_data as 'Data/StockPriceData.csv'
df_stock_price_data.to_csv('Data/StockPriceData.csv')